##**# Step 1: Load Dataset**

In [ ]:
import pandas as pd
import re
# Load dataset
file_path = "/content/sample_data/ai-medical-chatbot.csv"
df = pd.read_csv(file_path)

# Display first few rows
df.head()

## **# Step 2: Text** **Preprocessing**

In [ ]:
import numpy as np
# Function to clean text
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9 ]', '', text.lower())

# Apply text cleaning
df['Patient'] = df['Patient'].astype(str).apply(clean_text)
df['Doctor'] = df['Doctor'].astype(str).apply(clean_text)

# Display cleaned text
df.head()

## **# Step 3: Tokenization and Padding**

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenizing the patient (input) text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Patient'])
X = pad_sequences(tokenizer.texts_to_sequences(df['Patient']), padding='post')

# Tokenizing the doctor (output) text
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(df['Doctor'])
y_sequences = y_tokenizer.texts_to_sequences(df['Doctor'])

# Convert output sequences to categorical labels
y = np.array([seq[0] if seq else 0 for seq in y_sequences])

# Display tokenized output
X.shape, y.shape

## **# Step 4: Train-Test Split**

In [ ]:
from sklearn.model_selection import train_test_split
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display dataset sizes
X_train.shape, X_test.shape, y_train.shape, y_test.shape


## **# Step 5: Define and Train the Model**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
# Function to create the model
def create_model():
    model = Sequential([
        Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=X.shape[1]),
        LSTM(128),
        Dense(len(y_tokenizer.word_index) + 1, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create model
model = create_model()

# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=16)

# Save model
model.save('/mnt/data/health_chatbot.h5')

##**#Step 6: Deploy Chatbot using Flask**

In [ ]:
from flask import Flask, request, jsonify
import numpy as np
# Initialize Flask app
app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json['question']
    seq = pad_sequences(tokenizer.texts_to_sequences([clean_text(user_input)]), maxlen=X.shape[1], padding='post')
    prediction = model.predict(seq)
    response_index = np.argmax(prediction)
    response = y_tokenizer.index_word.get(response_index, 'Sorry, I am not sure.')
    return jsonify({'response': response})

# Run the app
if __name__ == '__main__':
    app.run(debug=True)
